# Inaccurate testing

In this notebook, we relax the accurate testing assumption by accounting for the possibility of false positives and negatives in antigen testing. To do this, we incorporate two additional parameters into our model:

- The true positive rate (antigen test sensitivity)
- The true negative rate (antigen test specificity)

These are parameters are estimated within the model, and are pinned down by informative priors calibrated with data from the [Joint PHE Porton Down & University of Oxford SARS-CoV-2 test development and validation cell](https://www.ox.ac.uk/sites/files/oxford/media_wysiwyg/UK%20evaluation_PHE%20Porton%20Down%20%20University%20of%20Oxford_final.pdf). 

In [1]:
import time
import arviz
import numpy as np
import pandas as pd
import stan
import nest_asyncio
nest_asyncio.apply()

In [2]:
# Parameters
P = 10
N = 3*(10 ** 4)
SEED = 1729

In [3]:
# Ground truth
np.random.seed(SEED)
true_transmission_rate = np.random.beta(2, 10, P)
true_occurrence_rate = np.random.beta(2, 10, P)
base_rate = np.random.beta(2, 10, 1)

test_sensitivity = np.random.beta(4, 3, 1)  # True positive rate
test_specificity = np.random.beta(50, 2, 1)  # True negative rate
true_lambda = np.array([test_sensitivity, test_specificity])

In [4]:
# Simulate data
data = {}
for p in range(P):
    occurrence = np.random.binomial(1, true_occurrence_rate[p], N)
    transmission = occurrence * np.random.binomial(1, true_transmission_rate[p], N)
    data[f'O{p+1}'] = occurrence
    data[f'T{p+1}'] = transmission

data['T0'] = np.random.binomial(1, base_rate, N)
X = pd.DataFrame(data)
z = X.loc[:, X.columns.str.startswith('T')].sum(axis=1)
y = (z > 0).astype(int)

# Introducing false positives and negatives
y = y*np.random.binomial(1, true_lambda[0], N) + (1-y)*np.random.binomial(1, (1-true_lambda[1]), N)

X = X.loc[:, X.columns.str.startswith('O')]
X.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   O1      30000 non-null  int64
 1   O2      30000 non-null  int64
 2   O3      30000 non-null  int64
 3   O4      30000 non-null  int64
 4   O5      30000 non-null  int64
 5   O6      30000 non-null  int64
 6   O7      30000 non-null  int64
 7   O8      30000 non-null  int64
 8   O9      30000 non-null  int64
 9   O10     30000 non-null  int64
dtypes: int64(10)
memory usage: 2.3 MB


In [5]:
# Create model
model_code = """
data {
  int<lower=0> N;                            // number of test observations
  int<lower=0> P;                            // number of places
  int<lower=0, upper=1> X[N,P];              // activity occurrences of tested individuals
  int<lower=0, upper=1> y[N];                // transmission (tested positive)
  real<lower=0, upper=1> mean_lambda[2];     // mean TP and TN test rates (for strong priors on tests)
  real<lower=0> se_lambda[2];                // standard error of test rates (for strong priors on tests)
}
parameters {
  real<lower=0, upper=1> theta[P];           // transmission rates
  real<lower=0, upper=1> rho;                // underlying risk
  real<lower=0, upper=1> lambda[2];          // True positive and true negative rates of tests [TP,TN]
}
model {
  // Precomputation
  real log1m_theta[P];
  real log1m_rho;
  real log_lambda[2];
  real log1m_lambda[2];
  real a_lambda[2];
  real b_lambda[2];
  
  for (p in 1:P) {
    log1m_theta[p] = log1m(theta[p]);
  }
  for(i in 1:2){
    log_lambda[i] = log(lambda[i]);
    log1m_lambda[i] = log1m(lambda[i]);
    
    a_lambda[i] = (((1-mean_lambda[i])/se_lambda[i]^2)-(1/mean_lambda[i]))*(mean_lambda[i]^2);
    b_lambda[i] = a_lambda[i]*((1/mean_lambda[i])-1);
  }
  
  log1m_rho = log1m(rho);
  
  // Priors
  theta ~ uniform(0, 1);
  rho ~ uniform(0, 1);
  lambda[1] ~ beta(a_lambda[1], b_lambda[1]);
  lambda[2] ~ beta(a_lambda[2], b_lambda[2]);

  // Likelihood
  for (n in 1:N) {
    real s = 0.0;
    for (p in 1:P) {
      if (X[n,p] == 1) {
        s += log1m_theta[p];
      }
    }
    s += log1m_rho;
    
    if (y[n] == 1) {
      target += log_sum_exp((log1m_exp(s) + log_lambda[1]), (s + log1m_lambda[2]));
    } 
    else {
      target += log_sum_exp((s + log_lambda[2]), (log1m_exp(s) + log1m_lambda[1]));
    }
  }
}
""" 

In [6]:
# Set antigen test mean and std. error for TP and TN rates (for strong priors)

# True positive
mean_tp = 0.73000
se_tp = 0.04133

# True negative
mean_tn = 0.99680
se_tn = 0.00066

mean_rates = np.array([mean_tp, mean_tn])
se_rates = np.array([se_tp, se_tn])

In [7]:
# Build model
model_data = {'N': N, 'P': P, 
              'X': X.to_numpy(), 'y': y.to_numpy(),
              'mean_lambda':mean_rates, 'se_lambda':se_rates}
posterior = stan.build(model_code, data=model_data, random_seed=1)

Building... This may take some time.
Messages from stanc:
  The parameter lambda has no priors.
Done.


In [ ]:
# Set NUTS parameters
nuts_samples = 1000
nuts_burnin = 500
nuts_chains = 16

# Posterior sampling
start = time.time()
fit = posterior.sample(num_samples=nuts_samples, num_warmup=nuts_burnin, num_chains=nuts_chains)
print(f"{time.time() - start:.02f} seconds elapsed")

Sampling...
     0/24000 [>---------------------------]   0%  1 sec/0     
     1/24000 [>---------------------------]   0%  1 sec/26822 
     1/24000 [>---------------------------]   0% 3 secs/55803 
     1/24000 [>---------------------------]   0% 4 secs/81917 
     1/24000 [>---------------------------]   0% 5 secs/108854
     1/24000 [>---------------------------]   0% 6 secs/136775
     1/24000 [>---------------------------]   0% 7 secs/163647
     1/24000 [>---------------------------]   0% 8 secs/189552
     1/24000 [>---------------------------]   0% 10 secs/218173
     1/24000 [>---------------------------]   0% 11 secs/244051
     1/24000 [>---------------------------]   0% 12 secs/270589
     1/24000 [>---------------------------]   0% 13 secs/299875
     1/24000 [>---------------------------]   0% 14 secs/326353
     1/24000 [>---------------------------]   0% 15 secs/352409
     1/24000 [>---------------------------]   0% 16 secs/379283
     1/24000 [>---------------------

In [ ]:
# Evaluate fit with test and trace resample
arviz.plot_trace(fit, figsize=(8, 6));

In [ ]:
fit = fit.to_frame()
print("Inference on fitted model:")
fit.describe().T

In [ ]:
print('Ground truth:')
print('Setting-specific transmission rates: ', true_transmission_rate)
print('Base rate: ', base_rate)
print('True positive and negative rates: ', true_lambda)